In [2]:
import sympy as syp
from IPython.display import Math as displayMath
from SplitOct import *
syp.init_printing(use_unicode=True)
# from copy import copy
# import numpy as np

import sys
print('python     :', sys.version)
print('sympy      :', syp.__version__)
print('SplitOct   :', version())
# print('numpy      :', np.__version__)
print()
!jupyter --version

python     : 3.10.10 (main, Mar  5 2023, 22:26:53) [GCC 12.2.1 20230201]
sympy      : 1.9
SplitOct   : 0.13

Selected Jupyter core packages...
IPython          : 8.12.0
ipykernel        : 6.22.0
ipywidgets       : 8.0.6
jupyter_client   : 8.1.0
jupyter_core     : 5.3.0
jupyter_server   : 2.5.0
jupyterlab       : 3.6.3
nbclient         : 0.7.3
nbconvert        : 7.3.1
nbformat         : 5.8.0
notebook         : 6.5.4
qtconsole        : not installed
traitlets        : 5.9.0


In [5]:
dispmath = lambda x: display(displayMath(x))
def tex(inputobj):
    if type(inputobj) == SplitOctonion:
        return inputobj.__repr__()
    else:
        return syp.latex(inputobj)
i = syp.I
def sypHermConj(x):
    return syp.transpose(syp.conjugate(x))

def check_clifford_algebra(g_metric, Γ):
    N = len(Γ)
    matrix_dim = Γ[0].shape[0]
    identity_matrix = syp.eye(matrix_dim)
    correct = True
    for m in range(N):
        for n in range(N):
            LHS = Γ[m]@Γ[n] + Γ[n]@Γ[m]
            RHS = 2 * g_metric[m,n] * identity_matrix
            if LHS != RHS:
                correct = False
                print(f'Problem in (m,n)=({m},{n}).')
    if correct: print('This forms the Clifford algebra satisfiying the metric.')

In [ ]:
m_mass = syp.symbols('m', real=True)
dee = 4*[None]
ψ = 8*[None]
γ = 4*[None]

for n in range(4):
    dee[n] = syp.symbols(f'\partial_{n}', commutative=False)
    ψ[n]   = syp.symbols(f'psi_{n}', commutative=False)
    ψ[4+n] = syp.symbols(f'psi_{4+n}', commutative=False)

ψ_mat_dirac = syp.Matrix([
    [ ψ[4] + i*ψ[7]],
    [-ψ[6] + i*ψ[5]],
    [ ψ[3] + i*ψ[0]],
    [ ψ[1] + i*ψ[2]]])

# ψ_mat_dirac = Ω @ ψ_mat_dirac

# ψ_mat_dirac = 1/syp.sqrt(2) * syp.Matrix([
#     [ ( ψ[4] - ψ[3]) + i*(ψ[7] - ψ[0])],
#     [ (-ψ[6] - ψ[1]) + i*(ψ[5] - ψ[2])],
#     [ ( ψ[4] + ψ[3]) + i*(ψ[7] + ψ[0])],
#     [ (-ψ[6] + ψ[1]) + i*(ψ[5] + ψ[2])],
# ])

γ[0] = syp.Matrix([ # dirac basis
    [ 1,  0,  0,  0],
    [ 0,  1,  0,  0],
    [ 0,  0, -1,  0],
    [ 0,  0,  0, -1]])

γ[1] = syp.Matrix([
    [ 0,  0,  0,  1],
    [ 0,  0,  1,  0],
    [ 0, -1,  0,  0],
    [-1,  0,  0,  0]])

γ[2] = syp.Matrix([
    [ 0,  0,  0, -i],
    [ 0,  0,  i,  0],
    [ 0,  i,  0,  0],
    [-i,  0,  0,  0]])

γ[3] = syp.Matrix([
    [ 0,  0,  1,  0],
    [ 0,  0,  0, -1],
    [-1,  0,  0,  0],
    [ 0,  1,  0,  0]])

Ω = 1/syp.sqrt(2) * syp.Matrix([
    [ 1,  0, -1,  0,],
    [ 0,  1,  0, -1,],
    [ 1,  0,  1,  0,],
    [ 0,  1,  0,  1,]
])

for n in range(4):
    γ[n] = Ω @ γ[n] @ Ω.inv() # change to Weyl basis

check_clifford_algebra(syp.diag(1, -1, -1, -1), γ)

dee_slash = dee[0]*γ[0] + dee[1]*γ[1] + dee[2]*γ[2] + dee[3]*γ[3]

Dirac_op = i*(dee_slash) - m_mass*syp.eye(4)

# Dirac_eq_complex = Ω.inv() @ Dirac_op @ ψ_mat_dirac
Dirac_eq_complex = Dirac_op @ ψ_mat_dirac

# Dirac_eq = syp.Matrix(
    # list(syp.re(Dirac_op @ ψ_mat_dirac)) + \
    # list(syp.im(Dirac_op @ ψ_mat_dirac))
# )

Dirac_eq_re = (Dirac_eq_complex + syp.conjugate(Dirac_eq_complex))/2
for n in range(4):
    Dirac_eq_re = Dirac_eq_re.subs(syp.conjugate(dee[n]), dee[n])
    Dirac_eq_re = Dirac_eq_re.subs(syp.conjugate(ψ[n]), ψ[n])
    Dirac_eq_re = Dirac_eq_re.subs(syp.conjugate(ψ[4+n]), ψ[4+n])

Dirac_eq_im = (Dirac_eq_complex - syp.conjugate(Dirac_eq_complex))/(2*i)
for n in range(4):
    Dirac_eq_im = Dirac_eq_im.subs(syp.conjugate(dee[n]), dee[n])
    Dirac_eq_im = Dirac_eq_im.subs(syp.conjugate(ψ[n]), ψ[n])
    Dirac_eq_im = Dirac_eq_im.subs(syp.conjugate(ψ[4+n]), ψ[4+n])
Dirac_eq_re = syp.expand(Dirac_eq_re)
Dirac_eq_im = syp.expand(Dirac_eq_im)
# disp(syp.expand(Dirac_eq_re))

Dirac_eq = syp.Matrix(
    list(Dirac_eq_re) + \
    list(Dirac_eq_im)
)
disp(Dirac_eq)